In [1]:
import torch
from transformers import *

# Transformers has a unified API
# for 10 transformer architectures and 30 pretrained weights.
#          Model          | Tokenizer          | Pretrained weights shortcut
MODELS = [
    (BertModel,       BertTokenizer,       'bert-base-uncased'),
    (OpenAIGPTModel,  OpenAIGPTTokenizer,  'openai-gpt'),
    (GPT2Model,       GPT2Tokenizer,       'gpt2'),
    (CTRLModel,       CTRLTokenizer,       'ctrl'),
    (TransfoXLModel,  TransfoXLTokenizer,  'transfo-xl-wt103'),
    (XLNetModel,      XLNetTokenizer,      'xlnet-base-cased'),
    (XLMModel,        XLMTokenizer,        'xlm-mlm-enfr-1024'),
    (DistilBertModel, DistilBertTokenizer, 'distilbert-base-cased'),
    (RobertaModel,    RobertaTokenizer,    'roberta-base'),
    (XLMRobertaModel, XLMRobertaTokenizer, 'xlm-roberta-base'),
 ]

In [2]:
# Let's encode some text in a sequence of hidden-states using each model:
for model_class, tokenizer_class, pretrained_weights in MODELS:
    
    # Load pretrained model/tokenizer
    tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
    model = model_class.from_pretrained(pretrained_weights)

    # Encode text
    # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
    input_ids = torch.tensor([tokenizer.encode("Here is some text to encode", add_special_tokens=True)])  
    with torch.no_grad():
        last_hidden_states = model(input_ids)[0]  # Models outputs are now tuples

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [3]:
# Each architecture is provided with several class for fine-tuning on down-stream tasks, e.g.
BERT_MODEL_CLASSES = [
    BertModel, 
    BertForPreTraining, 
    BertForMaskedLM, 
    BertForNextSentencePrediction,
    BertForSequenceClassification, 
    BertForTokenClassification, 
    BertForQuestionAnswering
]

In [ ]:
# All the classes for an architecture can be initiated from pretrained weights for this architecture
# Note that additional weights added for fine-tuning are only initialized
# and need to be trained on the down-stream task
pretrained_weights = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)

for model_class in BERT_MODEL_CLASSES:
    
    # Load pretrained model/tokenizer
    model = model_class.from_pretrained(pretrained_weights)

    # Models can return full list of hidden-states & attentions weights at each layer
    model = model_class.from_pretrained(pretrained_weights,
                                        output_hidden_states=True,
                                        output_attentions=True)
    
    input_ids = torch.tensor([tokenizer.encode("Let's see all hidden-states and attentions on this text")])
    all_hidden_states, all_attentions = model(input_ids)[-2:]

    # Models are compatible with Torchscript
    model = model_class.from_pretrained(pretrained_weights, torchscript=True)
    traced_model = torch.jit.trace(model, (input_ids,))

    # Simple serialization for models and tokenizers
    model.save_pretrained('./directory/to/save/')  # save
    model = model_class.from_pretrained('./directory/to/save/')  # re-load
    tokenizer.save_pretrained('./directory/to/save/')  # save
    tokenizer = BertTokenizer.from_pretrained('./directory/to/save/')  # re-load


In [5]:
from transformers import pipeline

# Allocate a pipeline for sentiment-analysis
nlp = pipeline('sentiment-analysis')
nlp('We are very happy to include pipeline into the transformers repository.')

[{'label': 'NEGATIVE', 'score': 0.9457083344459534}]

In [7]:
nlp = pipeline('question-answering')
nlp({
    'question': 'What is the name of the repository ?',
    'context': 'Pipeline have been included in the huggingface/transformers repository'
})

{'score': 0.5135620997716472,
 'start': 35,
 'end': 59,
 'answer': 'huggingface/transformers'}

In [9]:
from transformers import pipeline

# Allocate a pipeline for sentiment-analysis
nlp = pipeline('fill-mask')
nlp('My <mask> ate my card.')

[{'sequence': '<s> My cat ate my card.</s>',
  'score': 0.05796924605965614,
  'token': 4758},
 {'sequence': '<s> My dog ate my card.</s>',
  'score': 0.042779482901096344,
  'token': 2335},
 {'sequence': '<s> My roommate ate my card.</s>',
  'score': 0.03260496258735657,
  'token': 25537},
 {'sequence': '<s> My dad ate my card.</s>',
  'score': 0.02303222194314003,
  'token': 4252},
 {'sequence': '<s> My mom ate my card.</s>',
  'score': 0.01962525211274624,
  'token': 3795}]

In [11]:
from transformers import pipeline

# Allocate a pipeline for sentiment-analysis
nlp = pipeline('text-generation')
nlp('My cat ate my card.')

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


[{'generated_text': "My cat ate my card. I was so upset. I was like, 'I'm not going"}]

In [13]:
from transformers import pipeline

# Allocate a pipeline for sentiment-analysis
nlp = pipeline('summarization')
nlp('My cat ate my card.')

Your min_length is set to 56, but you input_length is only 8. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 8. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


[{'summary_text': "My cat ate my card. My dog ate my credit card. It's not my fault. I'm not a bad person. My cat is a bad dog. I don't want to blame my cat for anything. I want to be a good person. I love my cat."}]